In [19]:
# Warning control
import warnings
warnings.filterwarnings('ignore')



# # Load environment variables
# from dotenv import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, PDFSearchTool, WebsiteSearchTool, ScrapeWebsiteTool, DirectoryReadTool, FileReadTool, CodeInterpreterTool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
import DataAcquisition, DataProcessing, BuildTradingStrategy, BackTest, MarketAnalysis# AgentRoles.py
from crewai import Agent
from langchain.tools import Tool
from crewai.tools import BaseTool
from pydantic import Field


In [20]:
files = {
    'agents': 'config/agents.yaml',
    'tasks' : 'config/tasks.yaml'
}
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']


In [21]:
codellm=LLM(model="ollama/codestral", base_url="http://localhost:11434")
llm = LLM(model="ollama/phi4", base_url="http://localhost:11434")
prollm=LLM(model="ollama/deepseek-r1:70b", base_url="http://localhost:11434")
minillm=LLM(model="ollama/nemotron-mini", base_url="http://localhost:11434")

In [22]:
alpha_vantage = AlphaVantageAPIWrapper()
#search = GoogleSerperAPIWrapper()

class AlphaVantageTool(BaseTool):
    name: str = "alpha_vantage"
    description: str = "Useful to query financial data using AlphaVantage API"
    alpha_vantage: AlphaVantageAPIWrapper = Field(default_factory=AlphaVantageAPIWrapper)

    def _run(self, query: str) -> str:
        """Call Alpha Vantage API and return results"""
        try:
            
            return self.alpha_vantage.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

class YahooFinanceTool(BaseTool):
    name: str = "yahoo_finance"
    description: str = "Useful to query financial data using Yahoo Finance API"

    def _run(self, query: str) -> str:
        """Call Yahoo Finance API and return results"""
        try:
            # Implement Yahoo Finance API call here
            # This is a placeholder, replace with actual implementation
            return f"Yahoo Finance API Result for {query}"
        except Exception as e:
            return f"Error performing search: {str(e)}"

class TwitterAPITool(BaseTool):
    name: str = "twitter_api"
    description: str = "Useful to query Twitter API for sentiment analysis"

    def _run(self, query: str) -> str:
        """Call Twitter API and return results"""
        try:
            # Implement Twitter API call here
            # This is a placeholder, replace with actual implementation
            return f"Twitter API Result for {query}"
        except Exception as e:
            return f"Error performing search: {str(e)}"


In [23]:
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource

# pdf_source = PDFKnowledgeSource(
#     file_paths=["Offline_Resources/Books/(SpringerBriefs in Economics) Gagari Chakrabarti, Chitrakalpa Sen (auth.) - Momentum Trading on the Indian Stock Market-Springer India (2013).pdf",
#                  #"Offline_Resources/Books/Igor Tulchinsky et al. - Finding Alphas_ A Quantitative Approach to Building Trading Strategies-Wiley (2020).pdf",
#                  #"Offline_Resources/Books/Ernest P. Chan - Quantitative trading _ how to build your own algorithmic trading business.-John Wiley & Sons (2021).pdf",
#                  #"Offline_Resources/Books/C. Kelliher - Quantitative Finance With Python_ A Practical Guide to Investment Management, Trading, and Financial Engineering (2022) - libgen.li.pdf",
#                  ],
#     validate_content=True,
#     base_dir="/Users/sml74mc/FinBot/FinBot/"
# )
research_agent = Agent(
    config=agents_config['researcher'],
    verbose=True,
    llm=llm,
    function_calling_llm = minillm,
    tools=[
        SerperDevTool(),
    ],
    #allow_code_execution=True,
    #code_execution_mode="unsafe"
)

consultant_agent = Agent(
    config=agents_config['Professor'],
    verbose=True,
    llm=llm,
    function_calling_llm = minillm,
    #knowledge_sources = [pdf_source]
    
)

manager = Agent(
    config=agents_config['Portfolio Manager'],
    verbose=True,
    llm=prollm,
    function_calling_llm = minillm,
    allow_delegation=True,
)

analyst = Agent(
    config=agents_config['Analyst'],
    verbose=True,
    llm=codellm,
    function_calling_llm = minillm,
    #allow_code_execution = True,
    allow_delegation = True
)

data_agent = Agent(
    config=agents_config['Data Acquisition Agent'],
    verbose=True,
    llm=llm,
    tools=[
        ScrapeWebsiteTool(),
        WebsiteSearchTool(),
        AlphaVantageTool(),  # Using the wrapped tool
        SerperDevTool(),
        #CodeInterpreterTool()
    ],
    #allow_code_execution=True,
    #code_execution_mode='unsafe'
)




In [24]:
# tools.py
from crewai.tools import BaseTool
from pydantic import Field
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper

class AlphaVantageTool(BaseTool):
    name: str = "alpha_vantage"
    description: str = "Useful to query financial data using AlphaVantage API"
    alpha_vantage: AlphaVantageAPIWrapper = Field(default_factory=AlphaVantageAPIWrapper)

    def _run(self, query: str) -> str:
        """Call Alpha Vantage API and return results"""
        try:
            return self.alpha_vantage.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

class YahooFinanceTool(BaseTool):
    name: str = "yahoo_finance"
    description: str = "Useful to query financial data using Yahoo Finance API"

    def _run(self, query: str) -> str:
        """Call Yahoo Finance API and return results"""
        try:
            # Implement Yahoo Finance API call here
            # This is a placeholder, replace with actual implementation
            return f"Yahoo Finance API Result for {query}"
        except Exception as e:
            return f"Error performing search: {str(e)}"


In [25]:
alpha_vantage_tool = AlphaVantageTool()
yahoo_finance_tool = YahooFinanceTool()
research_task = Task(
        config=tasks_config['suggest_analysis'],
        agent=consultant_agent,
        tools=[SerperDevTool(), CodeInterpreterTool()],
        context=[]
    )

data_acquisition_task = Task(
        config=tasks_config['fetch_multi_frequency_ohlc'],
        agent=data_agent,
        tools=[alpha_vantage_tool, yahoo_finance_tool],
        context=[research_task],
        async_execution=True
    )

sentiment_data_acquisition_task = Task(
        config=tasks_config['scrape_news_sentiment'],
        agent=data_agent,
        context=[research_task],
        async_execution=True
    )


distribute_portfolio_task = Task(
        config=tasks_config['distribute_portfolio'],
        agent=manager,
        tools=[CodeInterpreterTool()],
    )

analysis_task = Task(
    config=tasks_config['run_analysis'],
    agent=analyst
)

identify_strategy_task = Task(
    description=tasks_config['idetify_trading_strategies'],
    agent=research_agent,
    context=[analysis_task],
    human_input=True
)

    # Create and configure the crew
trading_crew = Crew(
        agents=[research_agent, data_agent, analyst, manager, consultant_agent],
        tasks=[research_task, data_acquisition_task, sentiment_data_acquisition_task, analysis_task, distribute_portfolio_task,identify_strategy_task],
        verbose=2,
        memory=True,
        share_crew=True,
        max_workers=4
    )

ValidationError: 2 validation errors for Task
tools.0
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value='ScrapeWebsiteTool', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
tools.1
  Input should be a valid dictionary or instance of BaseTool [type=model_type, input_value='TwitterAPITool', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

2025-02-14 16:16:39,991 - 13656928256 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.7s (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)'))))
2025-02-14 16:16:41,173 - 13656928256 - _common.py-_common:105 - INFO: Backing off send_request(...) for 1.6s (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)'))))
2025-02-14 16:16:43,287 - 13656928256 - _common.py-_common:105 - INFO: Backing off send_request(...) for 2.3s (requests.exceptions.SSLError: HTTPSConnectionPool(host='us.i.posthog.co

In [ ]:
input = {'capital':200000,'goal':'Steady Passive Income of 2% of the capital each month'}
trading_crew.kickoff(input=input)